# BERTopic Arxiv
- https://www.maartengrootendorst.com/blog/bertopic/
- https://maartengr.github.io/BERTopic/getting_started/quickstart/quickstart.html
- https://www.kaggle.com/code/maartengr/topic-modeling-arxiv-abstract-with-bertopic
- plus SBERT

%%capture - cell magic, %%capture , which captures the stdout/stderr of a cell. With this magic you can discard these streams or store them in a variable.

In [1]:
#!conda install -c conda-forge hdbscan -y

In [2]:
#!python3 -m pip install python-dev-tools --user --upgrade

In [3]:
#!pip3 install bertopic

In [4]:
#!pip3 install -U sentence-transformers

## SBERT
Sentense transformation and embedding

In [5]:
from sentence_transformers import SentenceTransformer, util

In [6]:
from bertopic import BERTopic

In [7]:
from nltk import tokenize

In [8]:
import spacy

In [9]:
import scipy.spatial

In [10]:
import nltk
from nltk.cluster import KMeansClusterer
import pandas as pd
import numpy as np
import nltk
#nltk.download('punkt')

In [11]:
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [12]:
import tqdm

## Get Data

We need to split the text into iterable segments (sentences), which will be vectorised (embedded). SBERT will search for similarity between the sentences to find the closest that will formulate the topics in BERTopic.<br>
We are choosing from pretrained multilingual models for unsupervised learning, but can train/tune our own, based on our samples (see the examples in SBERT MMulti and SBERT official site sbert.net)

In [13]:
# test corpus
corp = '''Jeg har sammen med mine to medstuderende Sofie Amalie Landt og Benjamin Aizen Kongshaug været i praktik hos konsulentfirmaet HBS Economics. Virksomheden blev stiftet i 2015, som Høj-bjerre Brauer Schultz ApS, og skiftede i 2020 navn til HBS Economics. I dag er virksomheden udeluk-kende ledet af de to partnere Andreas Højbjerre og Esben Schultz, som hver har ansvaret for virk-somhedens to afdelinger, data science og økonomi. Virksomheden har kontor i København og består af 5 fastansatte og et større antal studentermedhjælper og praktikanter fordelt mellem de to afde-linger.  HBS har en vision om at tilbyde den bedste økonomiske rådgivning og analyse i Norden, med det mål at kunne skabe et bedre samfund. Det vil de opnå ved at tilbyde analyse og rådgivning af høj kvalitet, så deres kunder kan træffe beslutninger, skabe forandringer og fastsætte nye agendaer. Før i tiden henvendte HBS sig primært til offentlige autoriteter og organisationer, men i 2020, efter et større eksternt konsulentstop i det offentlige, omlagde HBS deres strategi som betyder at de i dag primært tilbyder deres ydelser til private virksomheder, herunder Lundbeck, DFDS og Glad Fon-den.  HBS har udviklet et værktøj til at håndtere og udtrække struktureret information fra store mængder af ustrukturerede data for at skabe ny viden og værdi for deres kunder. Det er disse opgaver, som Data Science afdelingen varetager. Ved at anvende metoder som data crawling, statistisk tekstana-lyse og data mining på f.eks. alle online job opslag i Norden, kan de tilbyde et bredt udvalg af unikke data drevne analyser. I samarbejde med IDA har HBS valgt at investere og udvikle et nyt datadrevet projekt kaldet JobSpire. JobSpire er en jobportal, der skal gøre det nemmere for jobsøgende at in-spirere, og lade sig inspirere, af andre med en lignende karriere. JobSpire skal på sigt blive en separat virksomhed. Vi har i vores praktikforløb været en del af Data Science, og har arbejdet på et potenti-elt anbefalingssystem, som skal integreres i JobSpire.  2. Arbejdsopgaver og refleksion af disse i forhold til teorier og modeller  JobSpire er et nyopstartet projekt og er derfor stadig i udviklingsfasen, hvilket har betydet at vi fra start selv måtte undersøge mulighederne for at lave et anbefalingssystem, sætte et udviklingsmiljø op, samt finde midlertidig test data.   Amanda Juhl Hansen  
 4/8 Hele applikationen er bygget op i Python og til det har jeg kunne bruge min viden herom fra Data Science. Applikationen er desuden tilknyttet en Postgres og en Neo4j database. Med viden fra 2. semester omkring Docker og containerization havde jeg sammen med mine to medstuderende op-rettet en Docker container til hver af de to databaser. Postgres databasen er en er en midlertidig erstatning for den i JobSpire og Neo4j databasen skulle danne grundlag for vores anbefalingssystem. Til opsætning af disse databaser, brugte jeg min viden og forståelse om databaser, som jeg havde tilegnet mig på 1. semester.  En af mine første arbejdsopgaver lød på at undersøge den test data vi fandt. Det gjorde jeg for at få et indblik og en forståelse for dataens struktur. Forberedelsen inkluderede at udtrække data fra kolonner til nye dataframes, så vi kunne arbejde med dataen. Herefter skulle de hver især renses og transformeres, for at berige dataen mest muligt. Det var en længere og omdiskuteret proces. For at kunne forbedrede dataen har jeg brugt min viden fra studiet om data preparation og operationer på dataframes.   Fra studiet har jeg fået kendskab til Cypher Language og her lærte jeg at lave simple cypher queries. Det har været brugbart i forbindelse med vores Neo4j database, hvor en af mine andre opgaver var at lave mere komplekse queries til at udtrække forskellige anbefalinger og inspiration til karriereveje og jobskifte. Derudover har jeg også tilføjet og brugt plugins som APOC og GDS, som jeg også har anvendt og fået kendskab til gennem studiet. En af mine sidste arbejdsopgaver har været at skulle opsætte API’er som skal udstille endpoints fra applikationen til JobSpire. Kommunikation mellem fleres services gennem et API er viden jeg har tillært mig på 2. semester, og jeg har derfor kunnet bruge denne viden til arbejdsopgaven. Derudover har jeg undervejs i udviklingsprocessen udført linting af vores applikation, som jeg har fået viden og kendskab til gennem faget test, hvor jeg lærte om static testing.  3. Læringsmål  3.1 Viden   Den første uge blev jeg introduceret til begge afdelinger i virksomheden. Den afdeling jeg skulle være i, består af 3 fastansatte som hver har deres rolle og ansvarsområde i teamet. Desuden blev vi Amanda Juhl Hansen  
 5/8 introduceret til det projekt som jeg sammen med mine medpraktikanter skulle arbejde på under praktikken. Den første mandag i hver måned afholdes et møde for begge afdelinger, hvor alle for-klarer hvilke opgaver der arbejdes på og eventuelle udfordringer der måtte være i samme forbin-delse. Formålet med mødet var at sikre at alle var indforstået med hvilke igangværende opgaver, sørge for at deadlines og aftaler blev overholdt, samt planlægge fremadrettet. Dagligdagen i mit team bar også præg af sparing mellem udviklere i form af flere faglige diskussioner, i forbindelse med problemløsninger.  3.2 Færdigheder  Under praktikken har jeg og mine medpraktikanter arbejdet i sprints af to uger, for at følge den samme proces som resten af Data Science afdelingen. Derudover anvender de KanBan Board meto-den og vi valgte at gøre det samme. Vi havde et fast møde, efter et sprints afslutning, hvor vi defi-nerede og tilføjede den rette mængde Cards til et kommende sprint. Det betød at vi kunne struktu-rere vores sprint og ved at uddelegere opgaver, kunne vi også strukturere vores daglige arbejde. Jeg brugte derfor vores KanBan board effektivt og dagligt til at planlægge og få et overblik over hvor langt jeg var nået og hvad der manglede for at færdiggøre opgaven. Mine medpraktikanter og jeg var gode til at spare med hinanden og give feedback, som resulterede i en masse små rettelser henvendt mod forbedring eller refaktorering. Afhængig af hvor omfattende rettelserne var, blev den enten rettet med det samme, eller der blev defineret et nyt card med opgaven, så fokus forblev på den igangværende.  Jeg blev undervejs i praktikken konfronteret med flere problemstillinger, som jeg sammen med mine medpraktikanter skulle finde en løsning på. En af de problematikker var at vi havde et relativt lille datasæt og at en karrieresti blev repræsenteret af flere rækker. Det betød at jeg i de tilfælde hvor en række indeholdt NaN værdier, ikke blot kunne slette rækken. Hvis jeg slettede en række (et kar-rieretrin i en karrieresti), ville det give et misledende billede og hvis jeg slettede karrierestier, vil vores datasæt blive endnu mindre. Løsningen indebar derfor at lave en metode, der udfyldte tomme startdatoer med dags dato for 2 år siden og tomme slutdatoer blev udfyldt med dagsdato, hvis det var det nuværende job, for at kunne udregne, hvor lang tid man havde haft et job og samle det i et Amanda Juhl Hansen  
 6/8 interval. Det gav langt større værdi at erstatte disse NaN-værdier med en smule misvisende data, fremfor at slette flere karrierestier.1   Efter at have lagt alle joberfaringer i Graph database opstod en anden problematik. Personer med to erhvervserfaringer efterfulgt af hinanden, med samme jobtitel og jobområde, resulterede i at noden i databasen havde en relation til sig selv. Det skyldes at det er jobtitel og jobområde der differentierer joberfaringer. Løsningen var at jeg lavede en metode, der lagde varigheden af hvor længe man havde haft jobbet sammen fra den nuværende og forgående, og derefter slettede den foregående joberfaring, for at undgå at der blev skabt en relation til den samme joberfaring. Vi øn-skede kun at gøre dette, hvis det var joberfaringer der lå lige efter hinanden2.   Vi oplevede at vores danske oversættelse af jobtitler ikke var nok, da det viste sig at flere jobtitler i vores testdata ikke kun var på dansk eller engelsk. Problematikken var at hvis der ikke fandtes en dansk oversættelse i databasen, vil både FuzzyMatch og Semantic Search ikke kunne give et kvalifi-ceret bud. Derfor testede vi om en engelsk oversættelse af jobtitlerne ville give et bedre resultat, hvilket var tilfældet og vi valgte derfor at bruge denne løsning og implementere det i den nuværende metode.  3.3 Kompetencer  Da jeg på studiet primært har programmeret objektorienteret og haft data science som valgfag, havde jeg en smule kendskab til Python. I samarbejde med mine medstuderende i praktikken har jeg haft rig mulighed for at udvide disse kompetencer, eftersom Python også er trivielt til funktionel programmering. Samtidig har jeg kunnet bruge mine kompetencer om objekt orienteret program-mering og Object Relational Mapping til at udvikle og strukturere Python objekter med dets relati-oner. For at mappe disse Python objekter og relationer ind i Neo4j-databasen, har jeg brugt tid på at læse og forstå dokumentationen om Neomodel, som er et Python bibliotek der tillader Object Graph Mapping. I forbindelse med vores anbefalingssystem, har jeg lavet en række komplekse cy-pher queries, der skal bruges til at give job inspiration til brugerne. Jeg har bl.a. lavet en cypher  1 Se logbog d. 9.2.2022 2 Se logbog: d. 4.3.2022 – 7.3.2022 Amanda Juhl Hansen  
 7/8 query der ved brug af GDS finder de korteste ruter, fra ens nuværende jobposition til en fremtidig. Ruterne er sorteret efter popularitet. Derudover har jeg lavet en cypher query der udtrækker kendte menneskers karrierestier, indenfor en bestemt distance med udgangspunkt i ens nuværende jobpo-sition. De sorteres efter hvor mange jobskifte der er mellem nuværende job, og slutningen af den kendt persons karrieresti.  Igennem en længere proces af at rense og forbedrede data under praktikken, har jeg brugt forskel-lige NumPy-operationer, samt filter og map funktioner. Ved at implementere disse flere steder har jeg opnået viden og kompetencer indenfor iterative operationer på DataFrames, som jeg før kun havde i begrænset omfang fra studiet.  4. Refleksion over praktikforløbets udbytte for virksomheden samt for mig selv  Virksomheden har ikke før arbejdet med Graph databaser og derfor har de begrænset erfaring med disse. Derfor har vores praktikophold skabt stor værdi for virksomheden, da vi har kunnet undersøge hvilke muligheder der er for at implementere en Graph database, som anbefalingssystem til Job-Spire. Tilsvarende har jeg, som praktikant, fået en dybere indsigt i Graph databaser, både hvordan de kan anvendes til datahåndtering og som potentielt anbefalingssystem. Vi har formået at opfylde virksomheden ønsker til projektet, samt udvikle en fuldt fungerende applikation der på sigt kan blive implementeret i JobSpire1.  Set i bakspejlet var jeg og mine medstuderende ikke særligt integreret i data science afdelingens hverdag og møder. Mit indtryk er, at det primært skyldes at afdelingen er relativ lille med få udvik-lere, som varetager andre opgaver, og det faktum at vi arbejder på et selvstændigt og nyt projekt. Det har betydet at jeg og mine medpraktikanter har haft ansvaret for projektet og derfor også har skulle træffe nogle store beslutninger på egen hånd. Det har lært mig at stå til ansvar for egne be-slutninger og have begrundelse for de valg der blev truffet under udviklingen, samt hvad et godt og kommunikativt samarbejde er, da jeg har haft en tæt dialog og samarbejde med mine medstude-rende under praktikken. Det kunne have givet mere værdi og et større udbytte for virksomheden,  1 Se logbog: d. 9.3.2022 Amanda Juhl Hansen  
 8/8 hvis de havde valgt at være mere inde i processen, samt være med til at træffe beslutninger for at justere retningen af projektet.   Mit praktikforløb hos HBS har været med til at give mig en større indsigt i hvilket område af min profession, jeg ønsker at arbejde med i fremtiden. Det skyldes primært det store udbytte jeg har fået igennem mit arbejde med Python, NLP, GNN og Graph databaser som anbefalingssystem. Det har udviklet mine faglige kompetencer og dermed skabt en stigende interesse for dette.         
'''

In [14]:
# strip in sentences
sentences = nltk.sent_tokenize(corp)

# strip leading and trailing spaces
sentencies = [sentence.strip() for sentence in sentences]
sentences

['Jeg har sammen med mine to medstuderende Sofie Amalie Landt og Benjamin Aizen Kongshaug været i praktik hos konsulentfirmaet HBS Economics.',
 'Virksomheden blev stiftet i 2015, som Høj-bjerre Brauer Schultz ApS, og skiftede i 2020 navn til HBS Economics.',
 'I dag er virksomheden udeluk-kende ledet af de to partnere Andreas Højbjerre og Esben Schultz, som hver har ansvaret for virk-somhedens to afdelinger, data science og økonomi.',
 'Virksomheden har kontor i København og består af 5 fastansatte og et større antal studentermedhjælper og praktikanter fordelt mellem de to afde-linger.',
 'HBS har en vision om at tilbyde den bedste økonomiske rådgivning og analyse i Norden, med det mål at kunne skabe et bedre samfund.',
 'Det vil de opnå ved at tilbyde analyse og rådgivning af høj kvalitet, så deres kunder kan træffe beslutninger, skabe forandringer og fastsætte nye agendaer.',
 'Før i tiden henvendte HBS sig primært til offentlige autoriteter og organisationer, men i 2020, efter et s

In [15]:
len(sentences)

86

In [16]:
data = pd.DataFrame(sentences)
data.columns=['sentence']
data

,sentence
0,Jeg har sammen med mine to medstuderende Sofie...
1,"Virksomheden blev stiftet i 2015, som Høj-bjer..."
2,I dag er virksomheden udeluk-kende ledet af de...
3,Virksomheden har kontor i København og består ...
4,HBS har en vision om at tilbyde den bedste øko...
...,...
81,Det har lært mig at stå til ansvar for egne be...
82,Det kunne have givet mere værdi og et større u...
83,Mit praktikforløb hos HBS har været med til at...
84,Det skyldes primært det store udbytte jeg har ...


In [17]:
# alternative get test text
# from sklearn.datasets import fetch_20newsgroups
# text = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']


## Sentence Embedding

most important hyperparameters in BERTopic:
- language - "english" (model is "distilbert-base-nli-stsb-mean-tokens") or "multilingual" ("xlm-r-bert-base-nli-stsb-mean-tokens")
- top_n_words - number of words per topic to extract (opt 10-20)
- n_gram_range - the number of words in topic representation (New York is n_gram_range=(2, 2))
- min_topic_size - the minimum size of a topic (less - more topics; more fewer topics, default 10)
- nr_topics - the number of topics to reduced to, or auto
- low_memory -  to True for less memory used in computation, no probs
- calculate_probabilities

Make sure that the embeddings are a numpy array with shape: (len(docs), vector_dim) where vector_dim is the dimensionality of the vector embeddings. 

In [18]:
# Choose model, https://www.sbert.net/docs/pretrained_models.html 
# model = SentenceTransformer('distiluse-base-multilingual-cased-v2') # better than 2 in Danish, not in English (see tests in SBERT Multi)
# model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2') # comparable to 3
sentence_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2', device="cpu")
# sentence_model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')

In [19]:
# Create your own embeddings to apply later as a model
embeddings = sentence_model.encode(sentences, show_progress_bar=True)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
# Train our topic model using our pre-trained sentence-transformers embeddings from above
# topics, probs = topic_model.fit_transform(sentences, embeddings)

In [21]:
# Create topic model
topic_model = BERTopic(language="multilingual", calculate_probabilities=True, 
                       verbose=True, embedding_model=sentence_model,  min_topic_size=6)

In [22]:
# fit the model to the corpus
topics, probs = topic_model.fit_transform(sentences)


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2023-03-29 13:25:44,236 - BERTopic - Transformed documents to Embeddings
2023-03-29 13:25:49,495 - BERTopic - Reduced dimensionality
2023-03-29 13:25:49,527 - BERTopic - Clustered reduced embeddings


In [23]:
len(topic_model.get_topic_info())

3

In [24]:
def get_sentence_embeddings(sentence):
    embedding = sentence_model.encode([sentence])
    return embedding[0]

In [25]:
data['embeddings'] = data['sentence'].apply(get_sentence_embeddings)
data

,sentence,embeddings
0,Jeg har sammen med mine to medstuderende Sofie...,"[0.10439316, 0.22922207, -0.22167933, -0.05284..."
1,"Virksomheden blev stiftet i 2015, som Høj-bjer...","[-0.26674688, 0.034413755, -0.18169071, 0.1152..."
2,I dag er virksomheden udeluk-kende ledet af de...,"[-0.0933343, 0.124671414, -0.18407618, -0.0527..."
3,Virksomheden har kontor i København og består ...,"[-0.09658054, 0.3123514, -0.13841988, -0.07573..."
4,HBS har en vision om at tilbyde den bedste øko...,"[-0.24162601, 0.21646976, -0.31479874, -0.1944..."
...,...,...
81,Det har lært mig at stå til ansvar for egne be...,"[0.110346906, 0.087466456, -0.11196691, -0.023..."
82,Det kunne have givet mere værdi og et større u...,"[-0.038013376, 0.15966125, -0.25663814, -0.083..."
83,Mit praktikforløb hos HBS har været med til at...,"[0.01944973, 0.23123045, -0.27309445, -0.08802..."
84,Det skyldes primært det store udbytte jeg har ...,"[-0.3442768, -0.15640004, -0.13993436, -0.1019..."


In [26]:
data['embeddings'][1].shape

(384,)

## Dimensionality Reduction

#### UMAP

Uniform Manifold Approximation and Projection (__UMAP__) is a dimension reduction technique that can be used for visualisation similarly to t-SNE, but also for general non-linear dimension reduction. The algorithm is founded on three assumptions about the data (see https://umap-learn.readthedocs.io/en/latest/index.html)

In [27]:
from umap import UMAP

In [28]:
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

In [29]:
# Create topic model
topic_model = BERTopic(language="multilingual", calculate_probabilities=True, 
                       verbose=True, embedding_model=sentence_model,  min_topic_size=3,
                       umap_model=umap_model).fit(sentences)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2023-03-29 13:25:52,094 - BERTopic - Transformed documents to Embeddings
2023-03-29 13:25:53,564 - BERTopic - Reduced dimensionality
2023-03-29 13:25:53,575 - BERTopic - Clustered reduced embeddings


In [30]:
len(topic_model.get_topic_info())

9

In [31]:
# better alternative
import umap
umap_embeddings = umap.UMAP(n_neighbors=15, 
                            n_components=5, 
                            metric='cosine').fit_transform(embeddings)

#### PCA

In [32]:
from sklearn.decomposition import PCA

In [33]:
dim_model = PCA(n_components=5)

In [34]:
# Create topic model
topic_model = BERTopic(language="multilingual", calculate_probabilities=True, 
                       verbose=True, embedding_model=sentence_model,  min_topic_size=3,
                       umap_model=dim_model).fit(sentences)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2023-03-29 13:25:56,442 - BERTopic - Transformed documents to Embeddings
2023-03-29 13:25:56,472 - BERTopic - Reduced dimensionality
2023-03-29 13:25:56,481 - BERTopic - Clustered reduced embeddings


### Model Attributes

There are a number of attributes that you can access after having trained your BERTopic model:


Attribute	Description
- topics_	The topics that are generated for each document after training or updating the topic model.
- probabilities_	The probabilities that are generated for each document if HDBSCAN is used.
- topic_sizes_	The size of each topic
- topic_mapper_	A class for tracking topics and their mappings anytime they are merged/reduced.
- topic_representations_	The top n terms per topic and their respective c-TF-IDF values.
- c_tf_idf_	The topic-term matrix as calculated through c-TF-IDF.
- topic_labels_	The default labels for each topic.
- custom_labels_	Custom labels for each topic as generated through .set_topic_labels.
- topic_embeddings_	The embeddings for each topic if embedding_model was used.
- representative_docs_	The representative documents for each topic if HDBSCAN is used.


For example, to access the predicted topics for the first 10 documents, we simply run the following:

In [35]:
topic_model.topics_[:10]

[0, -1, 0, -1, -1, -1, 0, -1, 1, 1]

In [36]:
topic_model.topic_representations_

{-1: [('at', 0.10356889561696697),
  ('en', 0.08920247927175652),
  ('og', 0.08201368755286952),
  ('med', 0.0613359919144186),
  ('har', 0.059879814627035115),
  ('af', 0.05412836514221814),
  ('for', 0.05412836514221814),
  ('de', 0.04907072913127422),
  ('var', 0.04907072913127422),
  ('til', 0.04774671824167732)],
 0: [('og', 0.10083034195992224),
  ('at', 0.0755050658368856),
  ('det', 0.07513692151493698),
  ('et', 0.06972005437233149),
  ('med', 0.058966043981532525),
  ('for', 0.05691537153601721),
  ('af', 0.05312101343361607),
  ('den', 0.04964447612529807),
  ('vi', 0.04888550856408449),
  ('en', 0.047750391002002755)],
 1: [('jeg', 0.13330378632175857),
  ('har', 0.11417282769085307),
  ('og', 0.10613720206307604),
  ('som', 0.09017061113700581),
  ('til', 0.08032822572669286),
  ('viden', 0.0722115025550482),
  ('data', 0.07221014709134264),
  ('på', 0.07110619427503198),
  ('python', 0.06713580765684579),
  ('at', 0.06388890186198012)]}

### Visualise

In [37]:
topic_model.get_topic_info().head(5)

,Topic,Count,Name
0,-1,36,-1_at_en_og_med
1,0,32,0_og_at_det_et
2,1,18,1_jeg_har_og_som


In [38]:
len(probs)

86

In [39]:
# probabilities
topic_model.visualize_distribution(probs[80], min_probability=0.015)

In [40]:
# topic_model.get_topic(topic_model.get_topic_freq().iloc[1].Topic)

In [41]:
topic_model.get_topic(2)

False

In [42]:
# visualize
topic_model.visualize_barchart(top_n_topics=11, height=700)

In [43]:
# The first word in a topic fits best, how many more are also good (elbow method?)
topic_model.visualize_term_rank()

In [44]:
topic_model.visualize_term_rank(log_scale=True)

In [45]:
topic_model.visualize_topics(top_n_topics=11)

ValueError: zero-size array to reduction operation maximum which has no identity

## Clustering

#### KMeans
- https://medium.com/nerd-for-tech/extractive-text-summarization-using-sentence-transformer-and-kmeans-clustering-algorithm-e942a6b33860
- https://colab.research.google.com/drive/1ClTYut039t-LDtlcd-oQAdXWgcsSGTw9?usp=sharing#scrollTo=pGSwzo0Gjw6R

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
NUM_CLUSTERS=3
iterations=25
X = np.array(data['embeddings'].tolist())
kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=iterations,avoid_empty_clusters=True)
assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

In [ ]:
data['cluster']=pd.Series(assigned_clusters, index=data.index)
data['centroid']=data['cluster'].apply(lambda x: kclusterer.means()[x])
data

In [ ]:
# compute the distance between a sentence and a centroid
from scipy.spatial import distance_matrix

def distance_from_centroid(row):
    #type of emb and centroid is different, hence using tolist below
    return distance_matrix([row['embeddings']], [row['centroid'].tolist()])[0][0]

In [ ]:
data['distance_from_centroid'] = data.apply(distance_from_centroid, axis=1)
data

Insert chat here!

Create a summary:

1. Group sentences based on the cluster column.
2. Sort the group in ascending order based on the distance_from_centroid column and select the first row (sentence having least distance from the mean)
3. Sort the sentences based on their sequence in the original text.

In [ ]:
summary = ' '.join(data.sort_values('distance_from_centroid',ascending = True).groupby('cluster').head(1).sort_index()['sentence'].tolist())
summary

In [ ]:
# or
cluster_model = KMeans(n_clusters=5)
topic_model = BERTopic(hdbscan_model=cluster_model).fit(sentences)

#### HDBSCAN

In [ ]:
from hdbscan import HDBSCAN

In [ ]:
hdbscan_model = HDBSCAN(min_cluster_size=3, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
topic_model = BERTopic(hdbscan_model=hdbscan_model).fit(sentences)

In [ ]:
import hdbscan
cluster = hdbscan.HDBSCAN(min_cluster_size=15,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings)

In [ ]:
import umap

In [ ]:
from umap import UMAP

In [ ]:
# visualise clusters
import matplotlib.pyplot as plt

# Prepare data
umap_data = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
result = pd.DataFrame(umap_data, columns=['x', 'y'])
result['labels'] = cluster.labels_

# Visualize clusters
fig, ax = plt.subplots(figsize=(20, 10))
outliers = result.loc[result.labels == -1, :]
clustered = result.loc[result.labels != -1, :]
plt.scatter(outliers.x, outliers.y, color='#BDBDBD', s=0.05)
plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=0.05, cmap='hsv_r')
plt.colorbar()

#### Hierarchical Clustering

In [ ]:
# clustering
topic_model.visualize_hierarchy(top_n_topics=20, width=800)

## CountVectorizer
to extract the important words that form a topic

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer_model = CountVectorizer(ngram_range=(2, 2)) # , stop_words="danish"
topic_model = BERTopic(vectorizer_model=vectorizer_model).fit(sentences)

In [ ]:
topic_model.get_topic_info().head(11)

In [ ]:
# heat map - dependencies
topic_model.visualize_heatmap(n_clusters=3, top_n_topics=100)

## Dynamic Topics
Trump Tweeter example: https://colab.research.google.com/drive/1un8ooI-7ZNlRoK0maVkYhmNRl0XGK88f?usp=sharing#scrollTo=1dR2ckNK782p 

In [ ]:
# topics_over_time = topic_model.topics_over_time(docs=tweets, 
                                                timestamps=timestamps, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

In [ ]:
# topics over time
# topics_over_time = topic_model.topics_over_time(abstracts, topics, years)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20, width=900, height=500)

### Topic per Class
https://maartengr.github.io/BERTopic/getting_started/topicsperclass/topicsperclass.html

In [ ]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

data = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))
docs = data["data"]
targets = data["target"]
target_names = data["target_names"]
classes = [data["target_names"][i] for i in data["target"]]

In [ ]:
topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(docs)

In [ ]:
#topic per class (arxiv example)
topics_per_class = topic_model.topics_per_class(abstracts, topics, classes=categories)

In [ ]:
topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=10, width=900)

### c-TF-IDF
for topics representation, create single docs for each cluster

In [ ]:
docs_df = pd.DataFrame(data, columns=["Doc"])
docs_df['Topic'] = cluster.labels_
docs_df['Doc_ID'] = range(len(docs_df))
docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})

In [ ]:
# regularisation and importance
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count
  
tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m=len(data))

In [ ]:
# representation
def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topic'])
                     .Doc
                     .count()
                     .reset_index()
                     .rename({"Topic": "Topic", "Doc": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)
topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)

In [ ]:
# reduction
for i in range(20):
    # Calculate cosine similarity
    similarities = cosine_similarity(tf_idf.T)
    np.fill_diagonal(similarities, 0)

    # Extract label to merge into and from where
    topic_sizes = docs_df.groupby(['Topic']).count().sort_values("Doc", ascending=False).reset_index()
    topic_to_merge = topic_sizes.iloc[-1].Topic
    topic_to_merge_into = np.argmax(similarities[topic_to_merge + 1]) - 1

    # Adjust topics
    docs_df.loc[docs_df.Topic == topic_to_merge, "Topic"] = topic_to_merge_into
    old_topics = docs_df.sort_values("Topic").Topic.unique()
    map_topics = {old_topic: index - 1 for index, old_topic in enumerate(old_topics)}
    docs_df.Topic = docs_df.Topic.map(map_topics)
    docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})

    # Calculate new topic words
    m = len(data)
    tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m)
    top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)

topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)

upper was here: https://www.maartengrootendorst.com/blog/bertopic/

In [ ]:
# or
from bertopic.vectorizers import ClassTfidfTransformer

ctfidf_model = ClassTfidfTransformer()
topic_model = BERTopic(ctfidf_model=ctfidf_model).fit(sentences)

In [ ]:
# Two measures for word selection: bm25_weighting and reduce_frequent_word
ctfidf_model = ClassTfidfTransformer(bm25_weighting=True)
topic_model = BERTopic(ctfidf_model=ctfidf_model )

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
topic_model = BERTopic(ctfidf_model=ctfidf_model )

https://colab.research.google.com/drive/1ClTYut039t-LDtlcd-oQAdXWgcsSGTw9?usp=sharing#scrollTo=1NVY1fF0krI8

In [ ]:
# TF-IDF from https://maartengr.github.io/BERTopic/getting_started/embeddings/embeddings.html#custom-embeddings
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TF-IDF sparse matrix
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
vectorizer = TfidfVectorizer(min_df=5)
embeddings = vectorizer.fit_transform(docs)

# Train our topic model using TF-IDF vectors
topic_model = BERTopic(stop_words="english")
topics, probs = topic_model.fit_transform(docs, embeddings)

### Update Topics

In [ ]:
topic_model.update_topics(docs, n_gram_range=(1, 2))

### Reduce Topics

In [ ]:
topic_model.reduce_topics(docs, nr_topics=60)

### Search Topics

In [ ]:
similar_topics, similarity = topic_model.find_topics("database", top_n=5); similar_topics

In [ ]:
topic_model.get_topic(71)

## Save Model
Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved.

In [ ]:
# Save model
topic_model.save("my_model")

In [ ]:
# Load model
my_model = BERTopic.load("my_model")

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing#scrollTo=Eh5qp58Hp7Ua

More embedding types: https://colab.research.google.com/drive/18arPPe50szvcCp_Y6xS56H2tY0m-RLqv?usp=sharing#scrollTo=Khqzs5D0Ejda

Semi-supervised topic modelling: https://colab.research.google.com/drive/1bxizKzv5vfxJEB29sntU__ZC7PBSIPaQ?usp=sharing